In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("MAST30034 project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/09/14 15:56:05 WARN Utils: Your hostname, Zongchaos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.3.48 instead (on interface en0)
22/09/14 15:56:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/14 15:56:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sdf_all_property= spark.read.parquet('../data/raw/domain-website-data')

In [3]:
df_all_property = sdf_all_property.toPandas()

In [4]:
df_all_property

,property_id,street,suburb,state,postcode,latitude,longitude,price,bedrooms,bathrooms,parking,property_type,is_rural,land_size,land_unit,is_retirement,url
0,13671355,104/300 Swanston Street,MELBOURNE,VIC,3000,-37.810608,144.964584,$550,2,1,0,Apartment / Unit / Flat,False,0,m²,False,https://www.domain.com.au/104-300-swanston-str...
1,14868152,101/25-33 Wills Street,MELBOURNE,VIC,3000,-37.810776,144.956924,$400,1,1,0,Apartment / Unit / Flat,False,0,m²,False,https://www.domain.com.au/101-25-33-wills-stre...
2,15572564,2108/288 Spencer Street,MELBOURNE,VIC,3000,-37.813774,144.952087,$350,1,1,0,Apartment / Unit / Flat,False,0,m²,False,https://www.domain.com.au/2108-288-spencer-str...
3,15831027,2201/288 Spencer Street,MELBOURNE,VIC,3000,-37.813774,144.952087,$400,1,1,0,Apartment / Unit / Flat,False,0,m²,False,https://www.domain.com.au/2201-288-spencer-str...
4,15996004,502/5 Sutherland Street,MELBOURNE,VIC,3000,-37.811417,144.960846,$430 per week,1,1,0,Apartment / Unit / Flat,False,0,m²,False,https://www.domain.com.au/502-5-sutherland-str...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15255,16065590,5 Eurack Rd,BEEAC,VIC,3251,-38.171036,143.635600,UNDER APPLICATION,3,1,2,House,False,0,m²,False,https://www.domain.com.au/5-eurack-rd-beeac-vi...
15256,16064656,7 Quigley St,YINNAR,VIC,3869,-38.324482,146.327637,$380 per week,3,2,2,House,False,0,m²,False,https://www.domain.com.au/7-quigley-st-yinnar-...
15257,16062004,28 Monash Ave,OLINDA,VIC,3788,-37.853085,145.363983,$495 weekly,3,1,0,House,False,0,m²,False,https://www.domain.com.au/28-monash-ave-olinda...
15258,15998256,Guyatts Road,TOONGABBIE,VIC,3856,-38.068634,146.615500,$600,4,2,2,House,False,0,m²,False,https://www.domain.com.au/guyatts-road-toongab...


In [5]:
property_postcode = sdf_all_property.select('postcode').dropDuplicates().toPandas()

In [6]:
property_postcode.to_csv('../data/raw/property_postcode.csv', index=False)

In [7]:
import pandas as pd
import folium

In [8]:
df_train_postcode = pd.read_csv("../data/raw/train_with_postcode.csv")

In [9]:
import openrouteservice as ors


client = ors.Client(key='5b3ce3597851110001cf624856044b6ee25648c9918b1adf78eedc57')



In [10]:
df_all_property['nearest_train_station'] = ""
for i in range(0, 500):
    
    #get the postcode of the property and find all train stations in this postcode
    postcode = df_all_property['postcode'][i]
    train_station = df_train_postcode.loc[df_train_postcode['postcode'] == postcode].to_dict()

    lat = list(train_station["LATITUDE"].values())
    lon = list(train_station["LONGITUDE"].values())
    station_loc = [[df_all_property['longitude'][i], df_all_property['latitude'][i]]]
    
    #get the location of the stations
    for j in range(0, len(lat)):
        station_loc.append([lon[j], lat[j]])
    

    #get the distance and duration between the properties and stations
    matrix = client.distance_matrix(
    locations = station_loc,
    sources = list(range(1, len(station_loc))),
    destinations = [0],
    profile='foot-walking',
    metrics=['distance', 'duration'],
    )
    
    #get the shortest duration among all stations
    ind_station = matrix['durations'].index(min(matrix['durations']))
    nearest_station = list(train_station['STOP_NAME'].values())[ind_station]
    df_all_property['nearest_train_station'][i] = nearest_station


ApiError: 403 ({'error': 'Quota exceeded'})

In [13]:
df_all_property.to_csv('../data/raw/walking_shortest_station.csv', index=False)